In [ ]:
# 環境準備
!pip install langchain langchain-openai langchain-community openai numpy scikit-learn

In [ ]:
# 導入必要的模組
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

# 設定 API 金鑰（請替換為您的實際金鑰）
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# 初始化嵌入模型
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

print("嵌入模型初始化完成！")


## 探索語義相似性

### 生成文本向量

In [ ]:
# 準備測試文本
test_texts = [
    "人工智慧正在改變世界",
    "AI 技術革新了我們的生活方式", 
    "機器學習是 AI 的一個重要分支",
    "深度學習神經網路模擬人腦結構",
    "今天天氣很好，陽光明媚",
    "我喜歡在週末去公園散步",
    "數據科學家需要掌握統計學知識",
    "演算法優化可以提升系統效能"
]

# 生成嵌入向量
print("正在生成嵌入向量...")
embeddings = embeddings_model.embed_documents(test_texts)

print(f"成功生成 {len(embeddings)} 個嵌入向量")
print(f"每個向量的維度：{len(embeddings[0])}")

In [ ]:
# 將嵌入轉換為 numpy 陣列以便分析
embeddings_array = np.array(embeddings)
print(f"嵌入矩陣形狀：{embeddings_array.shape}")

### 計算相似度矩陣

In [ ]:
# 計算相似度矩陣
similarity_matrix = cosine_similarity(embeddings_array)

print("=== 語義相似度分析 ===")
print("文本列表：")
for i, text in enumerate(test_texts):
    print(f"{i}: {text}")

print("\n最相似的文本對：")
# 找出最相似的文本對（排除自己與自己的比較）
for i in range(len(test_texts)):
    for j in range(i+1, len(test_texts)):
        similarity = similarity_matrix[i][j]
        if similarity > 0.8:  # 高相似度閾值
            print(f"文本 {i} 和 {j}：相似度 {similarity:.3f}")
            print(f"  「{test_texts[i]}」")
            print(f"  「{test_texts[j]}」")
            print()

# 找出特定文本的最相關內容
target_index = 0  # "人工智慧正在改變世界"
similarities = similarity_matrix[target_index]
sorted_indices = np.argsort(similarities)[::-1][1:]  # 排除自己

print(f"與「{test_texts[target_index]}」最相關的文本：")
for i, idx in enumerate(sorted_indices[:3]):
    print(f"{i+1}. 相似度 {similarities[idx]:.3f}: {test_texts[idx]}")

## 視覺化

In [ ]:
!pip install matplotlib scikit-learn

### 設定參數

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib.font_manager as fm

# 使用 PCA 將 1536 維降到 2 維
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_array)

# 設定中文字體（如果環境支援）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 創建視覺化圖表
plt.figure(figsize=(12, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, s=100)

# 為每個點添加標籤
for i, txt in enumerate(test_texts):
    plt.annotate(f'{i}: {txt[:10]}...', 
                (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.8)

plt.title('文本嵌入的二維視覺化\n(使用 PCA 降維)', fontsize=14)
plt.xlabel(f'主成分 1 (解釋變異: {pca.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'主成分 2 (解釋變異: {pca.explained_variance_ratio_[1]:.1%})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
print(f"PCA 降維保留了 {sum(pca.explained_variance_ratio_):.1%} 的原始變異")

## 向量資料庫

In [ ]:
# 導入向量資料庫相關模組
from langchain.vectorstores import Chroma
from langchain.schema import Document
import chromadb
from chromadb.config import Settings

print("開始建立向量資料庫...")

### 建立文件

In [ ]:
# 創建測試文件
sample_documents = [
    Document(
        page_content="機器學習是人工智慧的一個重要分支，它使計算機能夠從數據中學習而無需明確編程。",
        metadata={"source": "ml_intro.txt", "category": "技術", "difficulty": "初級"}
    ),
    Document(
        page_content="深度學習使用多層神經網路來模擬人腦的決策過程，在圖像識別和自然語言處理方面取得了突破性進展。",
        metadata={"source": "dl_guide.txt", "category": "技術", "difficulty": "進階"}
    ),
    Document(
        page_content="數據預處理是機器學習流程中的關鍵步驟，包括數據清理、特徵選擇和標準化等操作。",
        metadata={"source": "data_prep.txt", "category": "技術", "difficulty": "中級"}
    ),
    Document(
        page_content="AI 倫理關注人工智慧系統的公平性、透明度和責任歸屬，確保技術發展符合人類價值觀。",
        metadata={"source": "ai_ethics.txt", "category": "倫理", "difficulty": "中級"}
    ),
    Document(
        page_content="自動駕駛汽車結合了電腦視覺、機器學習和感測器技術，代表了人工智慧在交通領域的重要應用。",
        metadata={"source": "autonomous_cars.txt", "category": "應用", "difficulty": "進階"}
    ),
    Document(
        page_content="自然語言處理讓機器能夠理解和生成人類語言，應用範圍從聊天機器人到機器翻譯都有涉及。",
        metadata={"source": "nlp_overview.txt", "category": "技術", "difficulty": "中級"}
    )
]

print(f"準備了 {len(sample_documents)} 個示例文件")
for i, doc in enumerate(sample_documents):
    print(f"{i+1}. [{doc.metadata['category']}] {doc.page_content[:30]}...")


### 建立一個向量資料庫

In [ ]:
# 配置 Chroma 資料庫
chroma_settings = Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma_db_demo"
)

# 建立向量資料庫
vector_store = Chroma.from_documents(
    documents=sample_documents,
    embedding=embeddings_model,
    persist_directory="./chroma_db_demo",
    collection_name="ai_knowledge_base"
)

print("向量資料庫建立完成！")
print(f"資料庫位置：./chroma_db_demo")
print(f"集合名稱：ai_knowledge_base")

# 檢查資料庫狀態
collection = vector_store._collection
print(f"文件數量：{collection.count()}")


### 向量資料庫檢索

In [ ]:
# 測試相似度搜尋
def test_similarity_search(query, k=3):
    print(f"\n=== 查詢：「{query}」 ===")
    
    # 執行相似度搜尋
    results = vector_store.similarity_search(query, k=k)
    
    print(f"找到 {len(results)} 個相關文件：")
    for i, doc in enumerate(results):
        print(f"\n{i+1}. [{doc.metadata['category']} - {doc.metadata['difficulty']}]")
        print(f"   來源：{doc.metadata['source']}")
        print(f"   內容：{doc.page_content}")
    
    return results

# 測試不同類型的查詢
test_queries = [
    "什麼是深度學習？",
    "如何處理數據？",
    "AI 的道德問題",
    "自駕車技術"
]

for query in test_queries:
    test_similarity_search(query, k=2)


### 帶分數的檢索

In [ ]:
# 帶分數的相似度搜尋
def test_similarity_search_with_score(query, k=3, score_threshold=0.7):
    print(f"\n=== 帶分數查詢：「{query}」 ===")
    
    # 執行帶分數的相似度搜尋
    results_with_scores = vector_store.similarity_search_with_score(query, k=k)
    
    filtered_results = []
    for doc, score in results_with_scores:
        # Chroma 使用距離分數，值越小表示越相似
        # 我們轉換為相似度分數（1 - 標準化距離）
        similarity_score = 1 / (1 + score)
        if similarity_score >= score_threshold:
            filtered_results.append((doc, similarity_score))
    
    print(f"找到 {len(filtered_results)} 個高相關文件（相似度 ≥ {score_threshold}）：")
    for i, (doc, similarity) in enumerate(filtered_results):
        print(f"\n{i+1}. 相似度：{similarity:.3f}")
        print(f"   [{doc.metadata['category']} - {doc.metadata['difficulty']}]")
        print(f"   內容：{doc.page_content[:80]}...")
    
    return filtered_results

# 測試帶分數的搜尋
test_similarity_search_with_score("機器學習演算法", k=4, score_threshold=0.8)
test_similarity_search_with_score("人工智慧的未來發展", k=4, score_threshold=0.7)

### 元資料過濾搜尋

向量資料庫的一個強大功能是能夠結合語義搜尋和元資料過濾：

In [ ]:
# 基於元資料的過濾搜尋
def filtered_search(query, filters=None, k=3):
    print(f"\n=== 過濾搜尋：「{query}」 ===")
    if filters:
        print(f"過濾條件：{filters}")
    
    # 執行過濾搜尋
    if filters:
        results = vector_store.similarity_search(query, k=k, filter=filters)
    else:
        results = vector_store.similarity_search(query, k=k)
    
    print(f"找到 {len(results)} 個符合條件的文件：")
    for i, doc in enumerate(results):
        print(f"\n{i+1}. [{doc.metadata['category']} - {doc.metadata['difficulty']}]")
        print(f"   來源：{doc.metadata['source']}")
        print(f"   內容：{doc.page_content[:60]}...")
    
    return results

# 測試不同的過濾條件
print("1. 只搜尋技術類文件：")
filtered_search("機器學習", filters={"category": "技術"})

print("\n2. 只搜尋初級難度的文件：")
filtered_search("人工智慧", filters={"difficulty": "初級"})

print("\n3. 搜尋進階技術文件：")
filtered_search("深度學習", filters={"category": "技術", "difficulty": "進階"})


## 動手實作、第一個向量資料庫

### 導入套件

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import os
import json
from datetime import datetime

### 建立向量資料庫知識

In [ ]:
class VectorKnowledgeBase:
    """完整的向量知識庫類別"""
    
    def __init__(self, collection_name="knowledge_base", persist_directory="./vector_db"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=100,
            separators=["\n## ", "\n### ", "\n\n", "\n", " ", ""]
        )
        self.vector_store = None
        self.document_count = 0
        
    def create_comprehensive_documents(self):
        """創建包含多種主題的綜合文件集"""
        documents_data = [
            {
                "content": """
# 機器學習基礎指南

## 什麼是機器學習
機器學習是人工智慧的一個分支，它使計算機能夠在不被明確編程的情況下從數據中學習。機器學習演算法建立基於訓練數據的數學模型，以便對新數據進行預測或決策。
## 機器學習的類型
### 監督式學習
監督式學習使用標記的訓練數據來學習輸入和輸出之間的映射關係。常見的監督式學習任務包括分類和回歸。

### 無監督式學習
無監督式學習從無標記的數據中發現隱藏的模式。聚類和降維是常見的無監督式學習任務。

### 強化學習
強化學習通過與環境互動來學習，系統通過試錯來最大化累積獎勵。
                """,
                "metadata": {
                    "source": "ml_basics.md",
                    "category": "機器學習",
                    "level": "入門",
                    "author": "AI Lab",
                    "created_date": "2024-01-15"
                }
            },
            {
                "content": """
# 深度學習與神經網路

## 神經網路基礎
人工神經網路是受生物神經系統啟發的計算模型。它由相互連接的節點（神經元）組成，每個連接都有權重。

## 深度學習的突破
深度學習使用多層神經網路（通常超過三層）來學習數據的複雜模式。近年來，深度學習在電腦視覺、自然語言處理和語音識別等領域取得了革命性的進展。

## 常見的深度學習架構
### 卷積神經網路 (CNN)
CNN 特別適合處理圖像數據，使用卷積層來檢測局部特徵。

### 循環神經網路 (RNN)
RNN 適合處理序列數據，如時間序列或自然語言文本。

### Transformer
Transformer 架構徹底改變了自然語言處理領域，是 GPT 和 BERT 等模型的基礎。
                """,
                "metadata": {
                    "source": "deep_learning.md",
                    "category": "深度學習",
                    "level": "進階",
                    "author": "Neural Network Research Group",
                    "created_date": "2024-02-01"
                }
            },
            {
                "content": """
# 數據科學與分析

## 數據科學流程
數據科學是一個跨學科領域，結合了統計學、電腦科學和領域專業知識來從數據中提取洞察。

### 數據收集
數據收集是數據科學的第一步，包括識別相關數據源、設計收集策略和確保數據品質。

### 數據預處理
數據預處理包括清理、轉換和準備數據以供分析。這通常是最耗時的步驟，但對最終結果的品質至關重要。

### 探索性數據分析
探索性數據分析（EDA）幫助數據科學家理解數據的結構、模式和異常。視覺化是 EDA 的重要工具。

### 模型建立與驗證
選擇適當的演算法、訓練模型並評估其性能是數據科學流程的核心。

## 數據視覺化
有效的數據視覺化能夠揭示數據中的模式，並幫助傳達分析結果。常用的工具包括 Matplotlib、Seaborn 和 Plotly。
                """,
                "metadata": {
                    "source": "data_science.md",
                    "category": "數據科學",
                    "level": "中級",
                    "author": "Data Analytics Team",
                    "created_date": "2024-01-20"
                }
            },
            {
                "content": """
# AI 倫理與社會影響

## 人工智慧倫理的重要性
隨著人工智慧技術的快速發展和廣泛應用，AI 倫理成為越來越重要的議題。我們需要確保 AI 系統的發展和部署符合人類價值觀和社會利益。

## 主要倫理議題
### 偏見與公平性
AI 系統可能會反映和放大訓練數據中的偏見，導致不公平的結果。確保演算法公平性是 AI 倫理的核心議題。

### 透明度與可解釋性
許多 AI 系統，特別是深度學習模型，被視為「黑盒子」。提高 AI 決策的透明度和可解釋性對建立信任至關重要。

### 隱私保護
AI 系統通常需要大量個人數據進行訓練，如何在利用數據的同時保護個人隱私是一個重大挑戰。

### 責任歸屬
當 AI 系統做出錯誤決策時，如何確定責任歸屬？這涉及法律、技術和倫理多個層面的考量。

## 負責任的 AI 發展
建立負責任的 AI 發展框架需要技術專家、政策制定者、倫理學家和社會各界的共同努力。
                """,
                "metadata": {
                    "source": "ai_ethics.md",
                    "category": "AI 倫理",
                    "level": "專業",
                    "author": "Ethics in AI Institute",
                    "created_date": "2024-02-10"
                }
            },
            {
                "content": """
# 自然語言處理技術

## NLP 概述
自然語言處理（NLP）是人工智慧的一個分支，專注於讓計算機理解、解釋和生成人類語言。NLP 結合了語言學、電腦科學和認知科學的知識。

## 核心技術
### 文本預處理
文本預處理包括分詞、詞性標註、命名實體識別等基本任務，為後續分析奠定基礎。

### 語言模型
語言模型能夠預測文本中下一個詞或評估文本序列的概率。近年來，大型語言模型如 GPT 系列展現了驚人的能力。

### 情感分析
情感分析可以識別文本中表達的情感傾向，廣泛應用於社交媒體監控、客戶回饋分析等領域。

### 機器翻譯
神經機器翻譯系統使用深度學習技術，在翻譯品質上取得了顯著提升。

## 應用領域
NLP 技術廣泛應用於搜尋引擎、聊天機器人、內容推薦、語音助理等產品和服務中。

## 挑戰與未來
儘管 NLP 取得了巨大進展，但在理解語言的語境、諷刺、文化背景等方面仍面臨挑戰。
                """,
                "metadata": {
                    "source": "nlp_guide.md",
                    "category": "自然語言處理",
                    "level": "中級",
                    "author": "NLP Research Lab",
                    "created_date": "2024-01-25"
                }
            }
        ]
        
        documents = []
        for doc_data in documents_data:
            doc = Document(
                page_content=doc_data["content"],
                metadata=doc_data["metadata"]
            )
            documents.append(doc)
        
        return documents
    
    def build_knowledge_base(self):
        """建立完整的向量知識庫"""
        print("開始建立向量知識庫...")
        
        # 1. 創建文件
        documents = self.create_comprehensive_documents()
        print(f"創建了 {len(documents)} 個源文件")
        
        # 2. 文件分割
        all_chunks = []
        for doc in documents:
            chunks = self.text_splitter.split_documents([doc])
            print(f"文件 '{doc.metadata['source']}' 分割成 {len(chunks)} 個片段")
            all_chunks.extend(chunks)
        
        print(f"總共分割成 {len(all_chunks)} 個文件片段")
        
        # 3. 建立向量資料庫
        self.vector_store = Chroma.from_documents(
            documents=all_chunks,
            embedding=self.embeddings,
            persist_directory=self.persist_directory,
            collection_name=self.collection_name
        )
        
        self.document_count = len(all_chunks)
        print(f"向量知識庫建立完成！共包含 {self.document_count} 個向量化片段")
        
        return self.vector_store
    
    def search(self, query, k=5, category_filter=None, level_filter=None):
        """智慧搜尋功能"""
        if not self.vector_store:
            raise ValueError("知識庫尚未建立，請先執行 build_knowledge_base()")
        
        print(f"\n=== 搜尋查詢：「{query}」 ===")
        
        # 構建過濾條件
        filters = {}
        if category_filter:
            filters["category"] = category_filter
        if level_filter:
            filters["level"] = level_filter
        
        if filters:
            print(f"過濾條件：{filters}")
            results = self.vector_store.similarity_search(query, k=k, filter=filters)
        else:
            results = self.vector_store.similarity_search(query, k=k)
        
        print(f"找到 {len(results)} 個相關結果：")
        
        for i, doc in enumerate(results):
            print(f"\n--- 結果 {i+1} ---")
            print(f"來源：{doc.metadata['source']}")
            print(f"類別：{doc.metadata['category']} | 級別：{doc.metadata['level']}")
            print(f"作者：{doc.metadata['author']}")
            print(f"內容摘要：{doc.page_content[:120]}...")
        
        return results
    
    def get_statistics(self):
        """獲取知識庫統計資訊"""
        if not self.vector_store:
            return "知識庫尚未建立"
        
        # 這裡我們模擬統計，實際實作可能需要查詢資料庫
        stats = {
            "total_documents": self.document_count,
            "collection_name": self.collection_name,
            "storage_location": self.persist_directory
        }
        
        return stats


### 實例化並建立知識庫

In [ ]:
# 實例化並建立知識庫
print("正在建立綜合向量知識庫...")
kb = VectorKnowledgeBase(collection_name="ai_comprehensive_kb")
vector_store = kb.build_knowledge_base()

## 測試檢索功能

### 測試基本搜尋功能

In [ ]:

print("=== 基本搜尋測試 ===")
kb.search("什麼是神經網路？", k=3)

### 測試分類過濾搜尋

In [ ]:
# 測試分類過濾搜尋
print("\n=== 分類過濾搜尋測試 ===")
kb.search("數據分析", k=3, category_filter="數據科學")

### 測試級別過濾搜尋

In [ ]:
# 測試級別過濾搜尋
print("\n=== 級別過濾搜尋測試 ===")
kb.search("機器學習", k=3, level_filter="入門")

### 測試複合過濾

In [ ]:
# 測試複合過濾
print("\n=== 複合過濾搜尋測試 ===")
kb.search("演算法", k=2, category_filter="深度學習", level_filter="進階")

### 測試語義理解能力

In [ ]:
# 測試語義理解能力
print("\n=== 語義理解能力測試 ===")
semantic_queries = [
    "AI 的道德問題",
    "如何確保人工智慧的公平性？",
    "深度神經網路的架構",
    "數據清理和準備",
    "語言模型的應用"
]

for query in semantic_queries:
    print(f"\n查詢：{query}")
    results = kb.search(query, k=2)

--- 

## 進階檢索功能實作

In [ ]:
class AdvancedVectorKnowledgeBase(VectorKnowledgeBase):
    """具有進階功能的向量知識庫"""
    
    def semantic_similarity_analysis(self, query, threshold=0.7):
        """語義相似度分析"""
        print(f"\n=== 語義相似度分析：「{query}」 ===")
        
        # 獲取查詢的嵌入向量
        query_embedding = self.embeddings.embed_query(query)
        
        # 執行帶分數的搜尋
        results_with_scores = self.vector_store.similarity_search_with_score(query, k=10)
        
        # 過濾高相似度結果
        high_similarity_results = []
        for doc, score in results_with_scores:
            # 轉換距離分數為相似度分數
            similarity = 1 / (1 + score)
            if similarity >= threshold:
                high_similarity_results.append((doc, similarity))
        
        print(f"找到 {len(high_similarity_results)} 個高相似度結果（≥ {threshold}）：")
        
        for i, (doc, similarity) in enumerate(high_similarity_results):
            print(f"\n{i+1}. 相似度：{similarity:.3f}")
            print(f"   類別：{doc.metadata['category']}")
            print(f"   來源：{doc.metadata['source']}")
            print(f"   內容：{doc.page_content[:100]}...")
        
        return high_similarity_results
    
    def multi_query_search(self, queries, k_per_query=3):
        """多查詢聚合搜尋"""
        print(f"\n=== 多查詢聚合搜尋 ===")
        print(f"查詢列表：{queries}")
        
        all_results = []
        seen_content = set()
        
        for query in queries:
            results = self.vector_store.similarity_search(query, k=k_per_query)
            for doc in results:
                # 去重：避免重複的文件片段
                content_hash = hash(doc.page_content)
                if content_hash not in seen_content:
                    seen_content.add(content_hash)
                    all_results.append(doc)
        
        print(f"聚合去重後共 {len(all_results)} 個結果：")
        
        for i, doc in enumerate(all_results[:8]):  # 只顯示前8個
            print(f"\n{i+1}. [{doc.metadata['category']}] {doc.metadata['source']}")
            print(f"   {doc.page_content[:80]}...")
        
        return all_results
    
    def category_overview(self):
        """知識庫分類概覽"""
        print(f"\n=== 知識庫分類概覽 ===")
        
        # 這裡我們使用模擬數據，實際應用中可以查詢資料庫獲取統計
        categories = ["機器學習", "深度學習", "數據科學", "AI 倫理", "自然語言處理"]
        
        for category in categories:
            results = self.vector_store.similarity_search(
                f"{category}相關內容", 
                k=100, 
                filter={"category": category}
            )
            print(f"{category}：{len(results)} 個文件片段")
        
        return categories




### 升級到進階知識庫

In [ ]:
# 升級到進階知識庫
advanced_kb = AdvancedVectorKnowledgeBase(collection_name="ai_comprehensive_kb")
advanced_kb.vector_store = kb.vector_store
advanced_kb.document_count = kb.document_count

### 測試功能

In [ ]:
# 測試進階功能
advanced_kb.semantic_similarity_analysis("人工智慧的倫理考量", threshold=0.75)

advanced_kb.multi_query_search([
    "機器學習演算法",
    "數據預處理",
    "模型評估"
], k_per_query=2)

advanced_kb.category_overview()